<a href="https://colab.research.google.com/github/Penguin2611/Aspect-based-summarization-of-reviews/blob/master/Text_Preprocessing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import nltk
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
## Tokenized Output
## object creation
# stop_words = set(stopwords.words('english'))
# file = open('/content/drive/My Drive/amazon_review.txt')

def preprocess(s):
### Data Reading
  # z = file.readlines()
  # s = ""
  # s = s.join(z)


  ### Splitting input lines
  ### Tokenizing line by line
  stop_words = set(stopwords.words('english'))
  l = WordNetLemmatizer()
  t = TweetTokenizer() 
  for c in string.punctuation:   ### Removing Punctuations
      if c == string.punctuation[13]:
        s = s.replace(c," ")
      if c != string.punctuation[6] and c != string.punctuation[12]:
        s= s.replace(c,"")

  res = s.splitlines()
  print(res) 
  text = [] 
  for i in res:
    p = t.tokenize(i) ### Tokenization
    text.append(p)
    text.append('\n')

  ## Flatting the list

  flat_list = []
  for sublist in text:
      for item in sublist:
          flat_list.append(item)

  tokens = [token.lower() for token in flat_list]
  tagged = nltk.pos_tag(tokens) ### POS Tagging 


  ### Combining list elements to string
  ### Tokenized output

  ### Lemmization
  lem_text =[]
  for c in tokens:
    tokenize_text = l.lemmatize(c,pos='v')
    tokenize_text = l.lemmatize(tokenize_text,pos='n')
    tokenize_text = l.lemmatize(tokenize_text,pos='a')
    lem_text.append(tokenize_text)

  ### Stopwords Removal
  stop_text = []
  for w in lem_text: 
      if w not in stop_words: 
          stop_text.append(w)
  print("Processed Tokens",stop_text,'\n')
  

  # s = " "
  # output = s.join(stop_text)
  # print(output)


In [24]:
# open input file: 
ifile = open('/content/drive/My Drive/Musical_Instruments_5.json') 
all_data = list()
for i, line in enumerate(ifile): 
    # convert the json on this line to a dict
    data = json.loads(line)
    # extract what we want
    text = data['reviewText']
    prodid = data['asin']
    # add to the data collected so far
    all_data.append([prodid, text])
# create the DataFrame
df = pd.DataFrame(all_data, columns=['Product ID','Review'])
## Group by product ID
groupby_Id = df['Review'].groupby(df['Product ID'])
## z = No. of products
z = len(list(groupby_Id))//100
index = 0
for j in range(z):
  t = len(list(groupby_Id)[j][1])
  # print(t)
  for i in range(t):
    ### Text Preprocessing of reviews for each Product
    preprocess(list(groupby_Id)[j][1][index])
    index += 1 
ifile.close()



["Not much to write about here but it does exactly what it's supposed to  filters out the pop sounds  now my recordings are much more crisp  it is one of the lowest prices pop filters on amazon so might as well buy it they honestly work the same despite their pricing"]
Processed Tokens ['much', 'write', 'exactly', 'suppose', 'filter', 'pop', 'sound', 'record', 'much', 'crisp', 'one', 'low', 'price', 'pop', 'filter', 'amazon', 'might', 'well', 'buy', 'honestly', 'work', 'despite', 'price', '\n'] 

["The product does exactly as it should and is quite affordable I did not realized it was double screened until it arrived so it was even better than I had expected As an added bonus one of the screens carries a small hint of the smell of an old grape candy I used to buy so for reminiscent's sake I cannot stop putting the pop filter next to my nose and smelling it after recording  DIf you needed a pop filter this will work just as well as the expensive ones and it may even come with a pleasing